AgeNet/
│
├── environment/            # layer 1 – world & map
│   ├── __init__.py
│   ├── base.py
│   ├── simple.py
│   ├── sumo.py
│   └── geometry.py
│
├── physics/                # layer 2 – physical state & motion
│   ├── __init__.py
│   ├── mobility.py
│   └── state.py
│
├── communication/          # layer 3 – network & interaction
│   ├── __init__.py
│   ├── channel.py
│   ├── radio.py
│   ├── neighbor_finder.py
│   └── reques_policy.py
│
├── learning/               # layer 4 – intelligence & control
│   ├── __init__.py
│   ├── brain.py
│   ├── hamiltonian.py
│   ├── radius_controller.py
│   └── state.py
│
├── core/
│   ├── __init__.py
│   ├── agent.py
│   └── simulator.py
│
├── analysis/               # post-processing & evaluation
│   ├── __init__.py
│   ├── metrics.py
│   ├── topology.py
│   └── visualization.py
│
├── experiments/
│   ├── __init__.py
│   └── exporters.py
│
│
└── main.py


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from openpyxl import Workbook
from typing import List, Tuple

from AgeNet import environments, physics, Agent, AgentSimulator, AgentsSimulator
from AgeNet import analyses, experiments

2026-02-03 06:08:09.670035: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-02-03 06:08:09.705266: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-02-03 06:08:09.756142: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770086289.813637  144054 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770086289.846321  144054 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2026-02-03 06:08:10.011166: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [2]:
L = 45                                  # The length of the simulation box
buildings_type = "random"               # Type of placement of buildings (random or regular)
num_buildings  = 10                     # Number of buildings in environment
num_streets    = 0                      # Number of streets in environment
manual_environment_parameters = [L, buildings_type, num_buildings, num_streets]

sumo_cfg    = "../../Sumo/SumoScenario/simulation.sumocfg"
sumo_binary = "/nix/store/gp4hbfbrxg4zzjs7g3nl7vy0rw4jmm2y-sumo-1.22.0/bin/sumo"
step_length = 0.1                       # 
use_gui     = False                     # 
seed        = None                      # 
sumo_environment_parameters = [sumo_cfg, sumo_binary, step_length, use_gui, seed]

N = 20                                  # Number of agents
Alphas = [-0.5, +0.3, 1.0, -1000]       # Hamiltonian constant coefficients
state_dim     = 3                       # Dimensions of the agent state for neural network input
action_dim    = 2                       # Neural network output dimensions (Q values)
learning_rate = 1e-5                    # Learning rate in model training
discount_rate = 0.98                    # The effect of future decisions on current decisions
batch_size    = 20                      # Selected package of replay_memory to perform training
steps_per_train = 10                    # The training function is called once every ... steps
Initialize_model_path = f'./Saved Model/model_H_best_weight.keras'
brain_parameters = [state_dim, action_dim, learning_rate, discount_rate, batch_size, Initialize_model_path]

absorption  = 0.5
base_delay  = 0.01
max_delay   = 1.0
communication_parameters = [absorption, base_delay, max_delay]

moving     = True                       # Enable the move function to agents
requesting = True                       # Enable the sending request function
training   = True                       # Enable each agent model training function
Functions  = [moving, requesting, training]



### Agents Simulator Senario

In [ ]:
environment = environments.SimpleEnvironment(L, buildings_type, num_buildings, num_streets)

Agents = []
for i in range(N): 
    physical_state = physics.PhysicalState(environment, f'{i}')
    movement       = physics.NonMarkovian(environment)
    agent = Agent(f'{i}', Alphas, brain_parameters, communication_parameters, physical_state, movement)
    Agents.append(agent)

simulator = AgentsSimulator(environment, Agents, Functions)
result_exporter = experiments.ResultExporter(environment)
visualizer = analyses.NetworkVisualizer(environment)


In [ ]:
AveragedExcel = Workbook(); del AveragedExcel['Sheet']
ParticleExcel = Workbook(); del ParticleExcel['Sheet']

for step in range(21):
    simulator.run(Agents, 10)

    # -------------------------------------------------------------------------------------------
    result_exporter.collect(step, Agents)
    visualizer.draw(Agents, step)
    print("\rStep: {}".format(step), end="")

anim = visualizer.camera.animate(interval= 120, repeat=True, repeat_delay= 500, blit=False)
anim.save('test.gif')
result_exporter.export_excel(AveragedExcel, ParticleExcel, 'test')
ParticleExcel.save(f"./Particle (seed = {0}).xlsx")
AveragedExcel.save(f"./Averaged (seed = {0}).xlsx")




Step: 20

### Agent Simulator Senario

In [ ]:
environment = environments.SimpleEnvironment(L, buildings_type, num_buildings, num_streets)


Agents = []
agents_simulator: List[AgentSimulator] = []
for i in range(N): 
    physical_state = physics.PhysicalState(environment, f'{i}')
    movement       = physics.NonMarkovian(environment)
    agent = Agent(f'{i}', Alphas, brain_parameters, communication_parameters, physical_state, movement)
    Agents.append(agent)

    agent_simulator = AgentSimulator(agent, Functions, environment.build_bounds)
    agents_simulator.append(agent_simulator)


result_exporter = experiments.ResultExporter(environment)

2026-02-02 05:27:51.336070: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [ ]:
AveragedExcel = Workbook(); del AveragedExcel['Sheet']
ParticleExcel = Workbook(); del ParticleExcel['Sheet']

for step in range(11):
    for i in range(len(Agents)):
        # print(step, i)
        agents_simulator[i].manualEnvChage(environment)
        agents_simulator[i].step(Agents, 2)

    # -------------------------------------------------------------------------------------------
    result_exporter.collect(step, Agents)
    print("\rStep: {}".format(step), end="")


result_exporter.export_excel(AveragedExcel, ParticleExcel, 'test')
ParticleExcel.save(f"./Particle (seed = {0}).xlsx")
AveragedExcel.save(f"./Averaged (seed = {0}).xlsx")




Step: 10

### Tests

In [ ]:
sumo_binary = "/nix/store/gp4hbfbrxg4zzjs7g3nl7vy0rw4jmm2y-sumo-1.22.0/bin/sumo"
env = environments.SumoEnvironment("../../Sumo/SumoScenario/simulation.sumocfg", sumo_binary, use_gui=False)
env.start()

T = 100

for t in range(T):
    env.step()
    ids = env.get_entity_ids()
    for vid in ids:
        pos = env.get_position(vid)
        # print(t, '\t', vid, '\t', pos)

env.close()

In [17]:
delta_H = -50

x = np.clip(-delta_H / 4, -20, 2)  # flip
x = np.clip(-delta_H * 4, -20, 2)  # send request
p = np.exp(x)
p



7.38905609893065